In [22]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import when,udf, expr, col, count, regexp_replace
from pyspark.sql.types import IntegerType

In [3]:
spark = SparkSession.builder \
        .appName("Fifa_Seer") \
        .getOrCreate()

In [4]:
df = spark.read.csv("players_20.csv", header=True, inferSchema=True)
df.show()

+---------+--------------------+-----------------+--------------------+---+----------+---------+---------+-----------+-------------------+-------+---------+---------+--------+----------------+--------------+------------------------+---------+-----------+-------------+-------------------+---------+------------------+--------------------+-------------+------------------+-----------+----------+--------------------+---------------+--------------------+----+--------+-------+---------+---------+------+---------+-----------+----------+-----------+--------+--------------+--------------------+------------------+-------------------+--------------------------+-----------------------+-----------------+---------------+-----------+-----------------+------------------+------------------+---------------------+---------------------+----------------+------------------+----------------+----------------+-------------+-------------+--------------+----------------+--------------------+----------------------

In [5]:
df2 = df.drop("sofifa_id","player_url","long_name","contract_valid_until","real_face","release_clause_eur")
df2.show()

+-----------------+---+----------+---------+---------+-----------+-------------------+-------+---------+---------+--------+----------------+--------------+------------------------+---------+-----------+-------------+-------------------+--------------------+-------------+------------------+-----------+----------+---------------+--------------------+----+--------+-------+---------+---------+------+---------+-----------+----------+-----------+--------+--------------+--------------------+------------------+-------------------+--------------------------+-----------------------+-----------------+---------------+-----------+-----------------+------------------+------------------+---------------------+---------------------+----------------+------------------+----------------+----------------+-------------+-------------+--------------+----------------+--------------------+-----------------------+---------------------+----------------+-------------------+-------------------+-----------------+---

In [6]:
df2.groupby("body_type").count().show()

+-------------------+-----+
|          body_type|count|
+-------------------+-----+
|            Shaqiri|    1|
|PLAYER_BODY_TYPE_25|    1|
|             Neymar|    1|
|         C. Ronaldo|    1|
|             Stocky| 1016|
|           Courtois|    1|
|               Lean| 6505|
|              Messi|    1|
|             Normal|10750|
|          Akinfenwa|    1|
+-------------------+-----+



In [7]:
#shaqiri->Normal
#PLAYER_BODY_TYPE_25->Normal
#Neymar->Lean
#C. Ronaldo -> Normal
#Courtois-> Normal
#Messi->Normal
#Akinfenwa->Stocky

df2 = df2.withColumn("body_type", 
                     when(df2['body_type']=='Shaqiri','Normal')
                     .when(df2['body_type']=='PLAYER_BODY_TYPE_25','Normal')
                     .when(df2['body_type']=='Neymar','Lean')
                     .when(df2['body_type']=='C. Ronaldo','Normal')
                     .when(df2['body_type']=='Courtois','Normal')
                     .when(df2['body_type']=='Messi','Normal')
                     .when(df2['body_type']=='Akinfenwa','Stocky')
                     .otherwise(df2['body_type']))

df2.show()

+-----------------+---+----------+---------+---------+-----------+-------------------+-------+---------+---------+--------+----------------+--------------+------------------------+---------+-----------+-------------+---------+--------------------+-------------+------------------+-----------+----------+---------------+--------------------+----+--------+-------+---------+---------+------+---------+-----------+----------+-----------+--------+--------------+--------------------+------------------+-------------------+--------------------------+-----------------------+-----------------+---------------+-----------+-----------------+------------------+------------------+---------------------+---------------------+----------------+------------------+----------------+----------------+-------------+-------------+--------------+----------------+--------------------+-----------------------+---------------------+----------------+-------------------+-------------------+-----------------+-------------

In [8]:
df2 = df2.filter(df2["player_positions"]!='GK')
df2.show()

+-----------------+---+----------+---------+---------+-----------+-------------------+-------+---------+---------+--------+----------------+--------------+------------------------+---------+-----------+-------------+---------+--------------------+-------------+------------------+-----------+----------+---------------+--------------------+----+--------+-------+---------+---------+------+---------+-----------+----------+-----------+--------+--------------+--------------------+------------------+-------------------+--------------------------+-----------------------+-----------------+---------------+-----------+-----------------+------------------+------------------+---------------------+---------------------+----------------+------------------+----------------+----------------+-------------+-------------+--------------+----------------+--------------------+-----------------------+---------------------+----------------+-------------------+-------------------+-----------------+-------------

In [9]:
strng = "gk_diving|gk_handling|gk_kicking|gk_reflexes|gk_speed|gk_positioning|goalkeeping_diving|goalkeeping_handling|goalkeeping_kicking|goalkeeping_positioning|goalkeeping_reflexes"
a = strng.split('|')
df2 = df2.drop(*a)
df2.show()

+-----------------+---+----------+---------+---------+-----------+-------------------+-------+---------+---------+--------+----------------+--------------+------------------------+---------+-----------+-------------+---------+--------------------+-------------+------------------+-----------+----------+---------------+--------------------+----+--------+-------+---------+---------+------+--------------------+------------------+-------------------+--------------------------+-----------------------+-----------------+---------------+-----------+-----------------+------------------+------------------+---------------------+---------------------+----------------+------------------+----------------+----------------+-------------+-------------+--------------+----------------+--------------------+-----------------------+---------------------+----------------+-------------------+-------------------+-----------------+-------------------------+------------------------+----+----+----+----+----+----+-

In [10]:
df2 = df2.drop("nation_jersey_number","loaned_from")

In [11]:
df2.show()

+-----------------+---+----------+---------+---------+-----------+-------------------+-------+---------+---------+--------+----------------+--------------+------------------------+---------+-----------+-------------+---------+--------------------+-------------+------------------+----------+---------------+----+--------+-------+---------+---------+------+--------------------+------------------+-------------------+--------------------------+-----------------------+-----------------+---------------+-----------+-----------------+------------------+------------------+---------------------+---------------------+----------------+------------------+----------------+----------------+-------------+-------------+--------------+----------------+--------------------+-----------------------+---------------------+----------------+-------------------+-------------------+-----------------+-------------------------+------------------------+----+----+----+----+----+----+----+----+----+----+----+----+----

In [12]:
df2 = df2.drop("nation_position")

In [21]:
df2.select([count(when(col(c).isNull(), c)).alias(c) for c in df2.columns]).show()

+----------+---+---+---------+---------+-----------+----+-------+---------+---------+--------+----------------+--------------+------------------------+---------+-----------+---------+---------+-----------+-------------+------------------+------+----+--------+-------+---------+---------+------+-------------+------------------+-------------------+--------------------------+-----------------------+-----------------+---------------+-----------+-----------------+------------------+------------------+---------------------+---------------------+----------------+------------------+----------------+----------------+-------------+-------------+--------------+----------------+--------------------+-----------------------+---------------------+----------------+-------------------+-------------------+-----------------+-------------------------+------------------------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
|short_name|age|dob|height_c

In [26]:
df2 = df2.withColumn("player_tags", regexp_replace(col("player_tags"), "#", ""))
df2.show(truncate=False)

+-----------------+---+----------+---------+---------+-----------+-------------------+-------+---------+---------+--------+----------------+--------------+------------------------+---------+-----------+-------------+---------+---------------------------------------------------------------------------------------------------------------------------+-------------+------------------+----------+----+--------+-------+---------+---------+------+---------------------------------------------------------------------------------------------------------------------------------------------------+------------------+-------------------+--------------------------+-----------------------+-----------------+---------------+-----------+-----------------+------------------+------------------+---------------------+---------------------+----------------+------------------+----------------+----------------+-------------+-------------+--------------+----------------+--------------------+----------------------

In [27]:
df2 = df2.withColumn("player_tags", regexp_replace(col("player_tags"), " (CPU AI Only)", ""))
df2.show(truncate=False)

+-----------------+---+----------+---------+---------+-----------+-------------------+-------+---------+---------+--------+----------------+--------------+------------------------+---------+-----------+-------------+---------+---------------------------------------------------------------------------------------------------------------------------+-------------+------------------+----------+----+--------+-------+---------+---------+------+---------------------------------------------------------------------------------------------------------------------------------------------------+------------------+-------------------+--------------------------+-----------------------+-----------------+---------------+-----------+-----------------+------------------+------------------+---------------------+---------------------+----------------+------------------+----------------+----------------+-------------+-------------+--------------+----------------+--------------------+----------------------

In [29]:
df2 = df2.dropna(subset=['team_position','team_jersey_number'])
df2 = df2.fillna('not_specified')
df2.select([count(when(col(c).isNull(), c)).alias(c) for c in df2.columns]).show()

+----------+---+---+---------+---------+-----------+----+-------+---------+---------+--------+----------------+--------------+------------------------+---------+-----------+---------+---------+-----------+-------------+------------------+------+----+--------+-------+---------+---------+------+-------------+------------------+-------------------+--------------------------+-----------------------+-----------------+---------------+-----------+-----------------+------------------+------------------+---------------------+---------------------+----------------+------------------+----------------+----------------+-------------+-------------+--------------+----------------+--------------------+-----------------------+---------------------+----------------+-------------------+-------------------+-----------------+-------------------------+------------------------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
|short_name|age|dob|height_c

In [30]:
df2 = df2.drop("joined")
df2.show(truncate=False)

+-----------------+---+----------+---------+---------+-----------+-------------------+-------+---------+---------+--------+----------------+--------------+------------------------+---------+-----------+-------------+---------+---------------------------------------------------------------------------------------------------------------------------+-------------+------------------+----+--------+-------+---------+---------+------+---------------------------------------------------------------------------------------------------------------------------------------------------+------------------+-------------------+--------------------------+-----------------------+-----------------+---------------+-----------+-----------------+------------------+------------------+---------------------+---------------------+----------------+------------------+----------------+----------------+-------------+-------------+--------------+----------------+--------------------+-----------------------+---------

In [33]:
df2.select([count(when(col(c).isNull(),c)).alias(c) for c in df2.columns]).show()

+----------+---+---+---------+---------+-----------+----+-------+---------+---------+--------+----------------+--------------+------------------------+---------+-----------+---------+---------+-----------+-------------+------------------+----+--------+-------+---------+---------+------+-------------+------------------+-------------------+--------------------------+-----------------------+-----------------+---------------+-----------+-----------------+------------------+------------------+---------------------+---------------------+----------------+------------------+----------------+----------------+-------------+-------------+--------------+----------------+--------------------+-----------------------+---------------------+----------------+-------------------+-------------------+-----------------+-------------------------+------------------------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
|short_name|age|dob|height_cm|weigh

In [35]:
df2.coalesce(1).write.csv("players_20_processed.csv", header=True, mode='overwrite')